In [1]:
print('Begin')

Begin


In [2]:
'''For checking 3D space velocity against the grid values'''
'''Moved everything over from the XSEDE systemspec=pyfits.open('mcrx_200.fits')
CAMERA0=spec[25]
print(CAMERA0.header)
print(CAMERA0.header['CD1_1'])
Ang=[x[0]*10**(10) for x in spec[5].data]#'''

import sys 
sys.path.append('/usr/local/lib/python2.7/site-packages/ppxf')
print(sys.path)
import pyfits
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import WMAP9 as cosmo
from astropy.convolution import Gaussian1DKernel
from astropy.convolution import convolve
from ppxf import ppxf
import ppxf_util as util
from matplotlib.colors import LogNorm
import glob
from time import clock

import os

#spec=pyfits.open('q0.5_fg0.3_allrx10_sunruns/mcrx_200.fits')
#My\ Passport\ for\ Mac/
print(os.getcwd())
#Below is the older version (pre absorption lines):
#os.chdir(os.path.expanduser('/Volumes/My Passport for Mac/LAURA_Sims/q0.5_fg0.3_allrx10_sunruns/test_Smodel_hires_testlrange2'))
os.chdir(os.path.expanduser('/Volumes/My Passport for Mac/LAURA_Sims/q0.5_fg0.3_allrx10_sunruns/hirestest_kin/'))
print(os.getcwd())

['', '/Users/beckynevin/anaconda/lib/python36.zip', '/Users/beckynevin/anaconda/lib/python3.6', '/Users/beckynevin/anaconda/lib/python3.6/lib-dynload', '/Users/beckynevin/anaconda/lib/python3.6/site-packages', '/Users/beckynevin/anaconda/lib/python3.6/site-packages/Sphinx-1.5.1-py3.6.egg', '/Users/beckynevin/anaconda/lib/python3.6/site-packages/aeosa', '/Users/beckynevin/anaconda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg', '/Users/beckynevin/anaconda/lib/python3.6/site-packages/IPython/extensions', '/Users/beckynevin/.ipython', '/usr/local/lib/python2.7/site-packages', '/usr/local/lib/python2.7/site-packages/ppxf']
/Volumes/My Passport for Mac/LAURA_Sims
/Volumes/My Passport for Mac/LAURA_Sims/q0.5_fg0.3_allrx10_sunruns/hirestest_kin


In [3]:
spec=pyfits.open('mcrx_200.fits')
print(spec[32].header)#spec[10] and spec[11] are camera0-parameters and camera1-

CAMERA0=spec[25]
print(CAMERA0.header)
print(CAMERA0.header['CD1_1'])
Ang=[x[0]*10**(10) for x in spec[5].data]#



XTENSION= 'IMAGE   '           / IMAGE extension                                BITPIX  =                  -64 / number of bits per data pixel                  NAXIS   =                    3 / number of data axes                            NAXIS1  =                  300 / length of data axis 1                          NAXIS2  =                  300 / length of data axis 2                          NAXIS3  =                   10 / length of data axis 3                          PCOUNT  =                    0 / required keyword; must = 0                     GCOUNT  =                    1 / required keyword; must = 1                     EXTNAME = 'CAMERA0-AUX'                                                         HDUVERS =                    1                                                  COMMENT This HDU contains the images of the auxiliary quantities.  The third dimCOMMENT ension runs over the different quantities, see the xxxSLICE keywords forCOMMENT  which is which.                

In [4]:
grid=pyfits.open('grid_200.fits')
print(grid['PARTICLEDATA'].header)



XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / 8-bit bytes                                    NAXIS   =                    2 / 2-dimensional binary table                     NAXIS1  =                47236 / width of table in bytes                        NAXIS2  =              2415221 / number of rows in table                        PCOUNT  =                    0 / size of special data area                      GCOUNT  =                    1 / one data group (required keyword)              TFIELDS =                   12 / number of fields in each row                   EXTNAME = 'PARTICLEDATA'       / name of this binary table extension            TTYPE1  = 'ID      '           / label for field                                TFORM1  = 'J       '           / format of field                                TZERO1  =           2147483648 / offset for unsigned integers                   TSCAL1  =                    1 / data ar

In [ ]:
parts=grid['PARTICLEDATA'].data
print(parts['mass'][10:])#kpc
print(parts['position'][10:])#kpc
print(parts['velocity'][10:])#kpc/yr
print(len(parts['velocity']))
vel_3d=np.sqrt(parts['velocity'][:10][0][0]**2+parts['velocity'][:10][0][1]**2+
     parts['velocity'][:10][0][2]**2)
print(vel_3d, 'kpc/yr')
print(vel_3d*(3.086e16/3.154e7), 'km/s')
print(parts['radius'][10:])
print(type(parts))
plt.clf()

kpc_arcmin=cosmo.kpc_proper_per_arcmin(0.03)#insert the redshift to get the kpc/arcmin scaling

kpc_arcsec=(kpc_arcmin.value/60)#

xs=[]
ys=[]
vel=[]
L_bol=[]
for i in range(len(parts['velocity'])):#parts['velocity']
    xs.append(parts['position'][i][0]/kpc_arcsec)#now in arcsec
    ys.append(parts['position'][i][1]/kpc_arcsec)#now in arcsec
    L_bol.append(parts['L_bol'][i])
    vel.append(-parts['velocity'][i][2]*(3.086e16/3.154e7))
plt.scatter(xs,ys,c=vel, s=2)
plt.colorbar()
plt.show()

plt.scatter(xs,ys,c=L_bol, s=2)
plt.colorbar()
plt.show()




[    41835.51547093     41835.51547093     41835.51547093 ...,
     13945.86856155  10562134.30794222  13605724.75407805]
[[ -4.99356694  -8.70636624  -0.15280274]
 [ -4.53839726 -31.66702908 -13.08952186]
 [  5.32408086  10.7917068    4.64798255]
 ..., 
 [  4.93337411   8.48448933   0.46738974]
 [ -4.93273342  -8.57674282  -0.4479779 ]
 [  4.92300699   8.5607751    0.45560369]]
[[ -2.68241238e-07   1.06286260e-07  -1.60019774e-07]
 [  2.75001269e-08  -5.92960260e-08  -3.06404861e-08]
 [  4.05112400e-08  -1.02671677e-07  -1.23385437e-07]
 ..., 
 [ -1.15895389e-08  -4.94253130e-08   1.84269682e-07]
 [  9.38295748e-08   1.56709358e-07  -8.56147022e-08]
 [  1.32837179e-08  -1.57664964e-07   1.19107217e-08]]
2415221
3.61399483854e-07

In [ ]:

plt.scatter(xs,ys,c=vel, vmin=-150, vmax=150,s=5, cmap='gist_rainbow_r', alpha=0.75)

plt.colorbar(label=r'Velocity [km s$^{-1}$]')
plt.xlim([-50,50])
plt.ylim([-50,50])
plt.show()

In [ ]:
print('done')

In [62]:
snaps='../../q0.5_fg0.3_allrx10/snapshot_200.hdf5'

import h5py
f = h5py.File(snaps, 'r')
print("Keys: %s" % f.keys())

for k in f.keys():print(f[k])
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)
    
set1=f['PartType0']
print(set1)


import h5py    # HDF5 support

fileName = "../../q0.5_fg0.3_allrx10/snapshot_200.hdf5"
f = h5py.File(fileName,  "r")
for item in f.attrs.keys():
    print(item + ":", f.attrs[item])
mr = f['PartType0']
print(np.shape(mr))
print(mr)
i00 = f['PartType1']
print("%s\t%s\t%s" % ("#", "mr", "I00"))
for i in range(len(mr)):
    print("%d\t%g\t%d" % (i, i, i))#, mr[i]))#, i00[i]))
#f.close()


Keys: KeysView(<HDF5 file "snapshot_200.hdf5" (mode r)>)
<HDF5 group "/Header" (0 members)>
<HDF5 group "/PartType0" (12 members)>
<HDF5 group "/PartType1" (5 members)>
<HDF5 group "/PartType2" (5 members)>
<HDF5 group "/PartType4" (7 members)>
<HDF5 group "/PartType5" (10 members)>
Header
PartType0
PartType1
PartType2
PartType4
PartType5
<HDF5 group "/PartType0" (12 members)>
(12,)
<HDF5 group "/PartType0" (12 members)>
#	mr	I00
0	0	0
1	1	1
2	2	2
3	3	3
4	4	4
5	5	5
6	6	6
7	7	7
8	8	8
9	9	9
10	10	10
11	11	11
